## Imports


In [ ]:
# import sys
# sys.path.append('/media/linux-stuff/gpt2-diff/scripts')

import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch import device
from torch.optim.lr_scheduler import LambdaLR

import math
import os
import pandas as pd

from scripts.config import gpt2config
from scripts.model import DiffusionLM, LMEmbedding, Denoiser, Decoding
from scripts.utils import (
    MyTokenizer, 
    get_next_log_filename, 
    save_checkpoint, 
    load_checkpoint,
    posterior_mean,
    rounding_weight,
    get_batch,
    finalize_tokens,
    reverse_diffusion_with_clamping,
    visualize_embeddings_2d,
    fwd_diffusion
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)
torch.set_float32_matmul_precision('high')

## Tokenizer

In [ ]:
tokenizer = MyTokenizer(max_len=13)
tokenizer.decode(tokenizer.encode("Hello, tiktoken is fast!"))

In [ ]:
config = gpt2config(n_vocab=tokenizer.n_vocab,n_embed=16,mlp_expansion=4,n_latent=512)
model = DiffusionLM(config).to(device)
print(f"Total Model parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")
print(config.n_vocab)

## Testing Initialization

In [ ]:
sample_input = "Once upon a time in a land far away, there lived a"
sample_tokens = tokenizer.encode(sample_input)
sample_input_ids = torch.tensor([sample_tokens], device=device)  # (1, sequence_length)
sample_time_step = torch.tensor([10], device=device)  # (1,)

In [ ]:
sample_input_ids.shape

In [ ]:
sample_output, sample_logits = model(sample_input_ids, sample_time_step)  # (1, sequence_length, n_embed)

token_ids = finalize_tokens(sample_output, model.embedding.embed.weight)
decoded_output = tokenizer.decode(token_ids.squeeze(0).tolist())
print("Decoded Text:",decoded_output)

## Forward Diffusion

In [ ]:
# noisy_input = fwd_diffusion(model.embedding(sample_input_ids), torch.tensor([1000], device=device), alphas)

# token_ids = finalize_tokens(noisy_input, model.embedding.embed.weight)
# decoded_output = tokenizer.decode(token_ids.squeeze(0).tolist())
# print("Decoded Text:",decoded_output)

## Loading Datasets

In [ ]:
import pandas as pd

# Load E2E dataset - extract text from 'ref' column
df = pd.read_csv('datasets/e2e-dataset/trainset.csv')
text = ' '.join(df['ref'].tolist())

print(f"Dataset length: {len(text)} characters")
print(f"Number of samples: {len(df)}")
print(f"First sample: {df['ref'][0]}")

In [ ]:
# from datasets import load_dataset
# import pandas as pd

# # Load ROCStories dataset from HuggingFace
# rocstories = load_dataset("mintujupally/ROCStories")

# print(f"Dataset: {rocstories}")
# print(f"Train samples: {len(rocstories['train'])}")
# print(f"First story: {rocstories['train'][0]['text']}")

# # Convert to pandas DataFrame with 'ref' column
# train_df = pd.DataFrame({'ref': rocstories['train']['text']})
# test_df = pd.DataFrame({'ref': rocstories['test']['text']})

# # Save to CSV files
# train_df.to_csv('datasets/rocstories_train.csv', index=False)
# test_df.to_csv('datasets/rocstories_test.csv', index=False)

# print(f"\nSaved {len(train_df)} training samples to datasets/ROCStories/rocstories_train.csv")
# print(f"Saved {len(test_df)} test samples to datasets/ROCStories/rocstories_test.csv")

In [ ]:
# Split into train and test
train_size = int(0.9 * len(df))
train_df = df[:train_size].reset_index(drop=True)
test_df = df[train_size:].reset_index(drop=True)

print(f"Train samples: {len(train_df)}, Test samples: {len(test_df)}")

# Pre-encode all sequences for training efficiency
print("\nEncoding training data...")
train_encoded = []
for idx, row in train_df.iterrows():
    encoded = tokenizer.encode(row['ref'], max_len=64)  # Use fixed sequence length
    train_encoded.append(encoded)
    if (idx + 1) % 5000 == 0:
        print(f"Encoded {idx + 1}/{len(train_df)} train samples")

print("\nEncoding test data...")
test_encoded = []
for idx, row in test_df.iterrows():
    encoded = tokenizer.encode(row['ref'], max_len=64)
    test_encoded.append(encoded)

# Convert to tensors
train_encoded = torch.tensor(train_encoded, dtype=torch.long)
test_encoded = torch.tensor(test_encoded, dtype=torch.long)

print(f"\nTrain encoded shape: {train_encoded.shape}")
print(f"Test encoded shape: {test_encoded.shape}")

In [ ]:
# Test batch
w_stack = get_batch('train', batch_size=4, sequence_length=64, train_encoded=train_encoded, test_encoded=test_encoded, device=device)
print(f"Batch shape: {w_stack.shape}")
print(f"First sequence decoded: {tokenizer.decode(w_stack[0].tolist())}")

## Training Loop


In [ ]:
# Training configuration
max_iters = 100000  
learning_rate = 3e-3
eval_iters = 1000  # Much fewer eval iterations (was 200!)
batch_size = 16  # Larger batch for better GPU utilization
sequence_length = 64
T = 1000
num_timestep_samples = 4  # Sample 8 timesteps per iteration for better gradient estimate



In [ ]:
# Fixed alpha schedule - simple sqrt schedule
t = torch.arange(0, T+1, device=device, dtype=torch.float32)
alpha_bars = 1 - torch.sqrt(t / T)  # Goes from ~0 to 1-sqrt(1)=0
alpha_bars = torch.clamp(alpha_bars, min=0.001, max=0.999)
alphas = torch.zeros(T+1, device=device) #alpha_0 to alpha_T
alphas[0] = alpha_bars[0]
alphas[1:] = alpha_bars[1:] / alpha_bars[:-1]
alphas = torch.clamp(alphas, min=0.001, max=0.999)

# Precompute sqrt terms for efficiency
sqrt_ab = torch.sqrt(alpha_bars)
sqrt_1mab = torch.sqrt(1 - alpha_bars)

print(f"Alpha bars range: [{alpha_bars.min():.4f}, {alpha_bars.max():.4f}]")
print(f"Alphas range: [{alphas.min():.4f}, {alphas.max():.4f}]")


In [ ]:
model = torch.compile(model, mode='max-autotune')  # or 'max-autotune' for more optimization


In [ ]:
optimizer_model = torch.optim.AdamW(model.parameters(), lr=learning_rate,weight_decay=0.0)
lr_lambda = lambda step: 1.0 - (step / float(max_iters))
scheduler_model = LambdaLR(optimizer_model, lr_lambda=lr_lambda)

#### Training Loop

In [ ]:
log_file = get_next_log_filename('logs')
print(f"Logging to: {log_file}")

with open(log_file, 'w') as f:
    f.write("Iteration,Total_Loss,Denoising_Loss,Posterior_Loss,Anchor_Loss,Rounding_Loss\n")

checkpoint_counter = 0

for it in range(0, max_iters):

    w = get_batch('train', batch_size, sequence_length, train_encoded=train_encoded, test_encoded=test_encoded, device=device)
    w_emb = model.embedding(w)

    x0 = w_emb + 0.1 * torch.randn_like(w_emb)
    total_loss = 0.0
    
    eps = torch.randn_like(x0)
    denoising_loss = 0.0
    for _ in range(num_timestep_samples):
        t_random = torch.randint(1, T+1, (batch_size,), device=device)
        t_idx = t_random
        sqrt_ab_t = sqrt_ab[t_idx].view(batch_size, 1, 1)
        sqrt_1mab_t = sqrt_1mab[t_idx].view(batch_size, 1, 1)
        
        xt = sqrt_ab_t * x0 + sqrt_1mab_t * eps
        x0_hat = model.denoiser(xt, t_random)
        x0_hat = torch.clamp(x0_hat, min=-10.0, max=10.0)
        denoising_loss += F.mse_loss(x0_hat, x0)
    
    denoising_loss = denoising_loss / num_timestep_samples + 1e-8
    total_loss += denoising_loss
    
    t_T = torch.full((batch_size,), T, device=device)
    xT = sqrt_ab[-1] * x0 + sqrt_1mab[-1] * eps
    x0_hat_T = model.denoiser(xT, t_T)
    x0_hat_T = torch.clamp(x0_hat_T, min=-10.0, max=10.0)
    mu_hat_T = posterior_mean(xT, x0, T, alpha_bars, alphas)
    posterior_loss = F.mse_loss(mu_hat_T, torch.zeros_like(mu_hat_T)) + 1e-8
    total_loss += posterior_loss
    
    xt_1 = sqrt_ab[1] * x0 + sqrt_1mab[1] * torch.rand_like(x0)
    x0_hat_1 = model.denoiser(xt_1, torch.ones(batch_size, device=device))
    x0_hat_1 = torch.clamp(x0_hat_1, min=-10.0, max=10.0)
    anchor_loss = F.mse_loss(x0_hat_1, w_emb) + 1e-8
    total_loss += anchor_loss

    logits = x0_hat_1 @ model.embedding.embed.weight.T
    logits = torch.clamp(logits, min=-100.0, max=100.0)
    rounding_loss = rounding_weight(it, max_iters) * (F.cross_entropy(logits.view(-1, config.n_vocab), w.view(-1)) + 1e-8)
    total_loss += rounding_loss
    
    if torch.isnan(total_loss) or torch.isinf(total_loss):
        print(f"\n{'='*70}")
        print(f"TRAINING STOPPED: NaN/Inf detected at iteration {it}")
        print(f"{'='*70}")
        print(f"Loss Diagnostics:")
        print(f"  Total Loss:     {total_loss.item() if not torch.isnan(total_loss) else 'NaN'}")
        print(f"  Denoising:      {denoising_loss.item()}")
        print(f"  Posterior:      {posterior_loss.item()}")
        print(f"  Anchor:         {anchor_loss.item()}")
        print(f"  Rounding:       {rounding_loss.item()}")
        print(f"\nModel Output Statistics:")
        print(f"  x0_hat range:   [{x0_hat.min().item():.2f}, {x0_hat.max().item():.2f}]")
        print(f"  logits range:   [{logits.min().item():.2f}, {logits.max().item():.2f}]")
        print(f"\nGradient Statistics:")
        total_norm = 0.0
        for p in model.parameters():
            if p.grad is not None:
                param_norm = p.grad.data.norm(2)
                total_norm += param_norm.item() ** 2
        total_norm = total_norm ** 0.5
        print(f"  Total grad norm: {total_norm:.4f}")
        print(f"{'='*70}\n")
        break
    
    optimizer_model.zero_grad(set_to_none=True)
    total_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
    optimizer_model.step()
    scheduler_model.step()

    with open(log_file, 'a') as f:
        f.write(f"{it},{total_loss.item():.6f},{denoising_loss.item():.6f},{posterior_loss.item():.6f},{anchor_loss.item():.6f},{rounding_loss.item():.6f}\n")

    if it % eval_iters == 0:
        print(f"Iter {it}: loss = {total_loss.item():.4f}, denoising = {denoising_loss.item():.4f}, posterior = {posterior_loss.item():.4f}, anchor = {anchor_loss.item():.4f}, rounding = {rounding_loss.item():.4f}")

    if it % 5000 == 0 and it > 0:
        checkpoint_name = f"training_ckpt_{checkpoint_counter % 2}"
        save_checkpoint(model, config, alpha_bars, T, checkpoint_name, save_individual=False)
        checkpoint_counter += 1

print(f"\nTraining complete! Logs saved to: {log_file}")

torch.Size([16, 64, 16]) torch.Size([16, 64, 16])
tensor(False, device='cuda:0')
tensor(1.0578, device='cuda:0', grad_fn=<MaxBackward1>)
torch.Size([16, 64, 16]) torch.Size([16, 64, 16])
tensor(False, device='cuda:0')
tensor(1.0578, device='cuda:0', grad_fn=<MaxBackward1>)
torch.Size([16, 64, 16]) torch.Size([16, 64, 16])
tensor(False, device='cuda:0')
tensor(1.0578, device='cuda:0', grad_fn=<MaxBackward1>)
torch.Size([16, 64, 16]) torch.Size([16, 64, 16])
tensor(False, device='cuda:0')
tensor(1.0578, device='cuda:0', grad_fn=<MaxBackward1>)
torch.Size([16, 64, 16]) torch.Size([16, 64, 16])
tensor(False, device='cuda:0')
tensor(1.0590, device='cuda:0', grad_fn=<MaxBackward1>)
torch.Size([16, 64, 16]) torch.Size([16, 64, 16])
tensor(False, device='cuda:0')
tensor(1.0590, device='cuda:0', grad_fn=<MaxBackward1>)
torch.Size([16, 64, 16]) torch.Size([16, 64, 16])
tensor(False, device='cuda:0')
tensor(1.0590, device='cuda:0', grad_fn=<MaxBackward1>)
torch.Size([16, 64, 16]) torch.Size([16, 

In [ ]:
with torch.no_grad():
    w = get_batch('train', 1, sequence_length)
    x0 = model.embedding(w)
    print(x0)
    eps = torch.randn_like(x0)

    for t in [1, T//4, T//2, T]:
        xt = sqrt_ab[t-1] * x0 + sqrt_1mab[t-1] * eps
        print(t, torch.norm(xt - x0).item())


## Inference

In [ ]:
context_length = 64
generated_tokens, generated_text = reverse_diffusion_with_clamping(
    model=model,
    config=config,
    tokenizer=tokenizer,
    alpha_bars=alpha_bars,
    T=T,
    context_length=context_length,
    batch_size=1,
    clamping_start=0.5,
    skip_step=10,
    display_at_steps=[T//2, 1],
    device=device
)

## Visualizing the Embedding Space

In [ ]:
my_vocab_itos_list = [tokenizer.decode([i]) for i in range(config.n_vocab-4) or range(config.n_vocab-3, config.n_vocab)]

In [ ]:
visualize_embeddings_2d(model.embedding.embed, my_vocab_itos_list[:3000], top_n=3000)

## Saving/Load Model Checkpoints

In [ ]:
# Save model checkpoint
save_checkpoint(
    model=model,
    config=config,
    alpha_bars=alpha_bars,
    T=T,
    checkpoint_name='E2E_v2',  # Customize as needed
    save_individual=True
)


#### Loading Pre-trained Model

In [ ]:
# Load model checkpoint
model, config, alpha_bars, T, sqrt_ab, sqrt_1mab = load_checkpoint(
    checkpoint_name='0.5k_12k_E2E',  # Update as needed
    device=device,
    eval_mode=True
)


### Parts of Speech Controller 

In [ ]:
controller_config = gpt2config(n_vocab=tokenizer.n_vocab,n_layer=4)

In [ ]:
class POSClassifier(nn.Module):
    def __init__(self,config,pos_vocab):
        super().__init__()
        self.config = config
        self.embedding = LMEmbedding(config)
        self.denoiser = Denoiser(config)
        self.decoder = nn.Linear(config.n_embed, pos_vocab)  # Binary classification
        
    def forward(self,input_ids,time_step):
        input_embeddings = self.embedding(input_ids)  # (B,T,C)
        x = self.denoiser(input_embeddings,time_step)  # (B,T,C)
        logits = self.decoder(x)  # (B,T,pos_vocab)
        
        return logits

In [ ]:
controller = POSClassifier(controller_config,pos_vocab=10).to(device)